## Computer -> Drone

In [3]:
import logging
import socket
import sys
import time

In [6]:
# 로깅설정

logging.basicConfig(level = logging.INFO, stream = sys.stdout)
logger = logging.getLogger(__name__)


In [15]:
# 드론을 관리하기 위한 클래스 

class DroneManager(object):
    def __init__(self, host_ip = '192.168.10.2', host_port = 8889,
                drone_ip = '192.168.10.1', drone_port = 8889):
        self.host_ip = host_ip
        self.host_port = host_port
        self.drone_ip = drone_ip
        self.drone_port = drone_port
        self.drone_address = (drone_ip, drone_port)
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.socket.bind((self.host_ip, self.host_port))
        self.socket.sendto(b'command', self.drone_address)
        self.socket.sendto(b'streamon',self.drone_address)
    
    # 클래스가 메모리에서 삭제될때 하는 메직메소드
    def __dell__(self):
        self.stop()
    
    # 클래스가 삭제된 경우 현재 열고 있던 소캣을 닫아준다.
    def stop(self):
        self.socket.close()
    
    # 이착륙과 같은 명령을 수향하기 위한 함수
    def send_command(self, command):
        # 로그 작성
        logger.info({'action' : 'send_command', 'command' : command}) 
        
        # 문자열로 명령어(command)가 들어오기 때문에 인코딩 후 통해 전달
        self.socket.sendto(command.encode('utf-8'),self.drone_address)
        
    def takeoff(self):
        self.send_command('takeoff')
        
    def land(self):
        self.send_command('land')

In [17]:
# Python 코드가 처음에 불려왔을 때 실행
if __name__ == '__main__':
    drone_manager = DroneManager()
    drone_manager.takeoff()
    
    time.sleep(10)
    
    drone_manager.land()

OSError: [WinError 10049] 요청한 주소는 해당 컨텍스트에서 유효하지 않습니다

## Drone -> Computer

In [18]:
import logging
import socket
import sys
import time

# 언제 드론에서 컴퓨터로 정보가 전달될지 모르기 때문에 스레딩 사용
import threading


In [29]:
# 드론을 관리하기 위한 클래스 

class DroneManager(object):
    def __init__(self, host_ip = '192.168.10.2', host_port = 8889,
                drone_ip = '192.168.10.1', drone_port = 8889):
        self.host_ip = host_ip
        self.host_port = host_port
        self.drone_ip = drone_ip
        self.drone_port = drone_port
        self.drone_address = (drone_ip, drone_port)
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.socket.bind((self.host_ip, self.host_port))
        
        # 반응에 대한 default 값
        self.response = None
        # 어떤 이벤트로 멈추고 싶을때 사용할 이벤트 설정
        self.stop_event = threading.Event()
        # 스레드가 돌릴 함수 : receive_response이며 인자는 args로 tuple의 형태로 stopevent를 전달
        self._response_thread = threading.Thread(target = self.receive_response, args = (self.stop_event,))
        # 스레드 실행
        self._response_thread.start()
        
        # 어떤 기능을 수행하기 위해서는 command라는 명령이 먼저 Drone에 전달 되어야한다.
        self.socket.sendto(b'command', self.drone_address)
        self.socket.sendto(b'streamon',self.drone_address)
        
    # 스레딩 돌릴 함수( 해당 함수는 stop_event가 아닌 경우 계속해서 돌면서 정보를 받는다)
    def receive_response(self, stop_event):
        while not stop_event.is_set():
            try:
                self.response, ip = self.socket.recvfrom(3000)
                logger.info({'action' : 'receive_response' , 'response' : self.response})
            except socket.error as ex:
                logger.error({'action' : 'receive_response', 'ex' : ex})
                break
                
    # 클래스가 메모리에서 삭제될때 하는 메직메소드
    def __dell__(self):
        self.stop()
    
    # 클래스가 삭제된 경우 현재 열고 있던 소캣을 닫아준다.
    def stop(self):
        # 드론이 멈춘다면 해당 스레드 또한 종료시켜라
        self.stop_event.set()
        
        # receive_response에 while문이 실행 중이라면 종료 후 실행시키기 위한 구문
        retry = 0
        while self._response_thread.isAlive():
            time.sleep(0.3)
            if retry > 30:
                break
            # 너무 오랜시간을 기다리지 않기 위한 retry 값 수정
            retry += 1
        
        self.socket.close()
    
    # 이착륙과 같은 명령을 수향하기 위한 함수
    def send_command(self, command):
        # 로그 작성
        logger.info({'action' : 'send_command', 'command' : command}) 
        
        # 문자열로 명령어(command)가 들어오기 때문에 인코딩 후 통해 전달
        self.socket.sendto(command.encode('utf-8'),self.drone_address)
        
        retry = 0
        while self.response is None:
            time.sleep(0.3)
            if retry > 3:
                break
            retry += 1
        
        if self.response is None:
            response = None
        else:
            response = self.response.deconde('utf-8')
        self.response = None
        return response
        
    def takeoff(self):
        # takeoff의 수행여부 판단을 위해 return을 사용
        return self.send_command('takeoff')
        
    def land(self):
        # land의 수행여부 판단을 위해 return을 사용
        return self.send_command('land')

In [24]:
# Python 코드가 처음에 불려왔을 때 실행
if __name__ == '__main__':
    drone_manager = DroneManager()
    drone_manager.takeoff()
    
    time.sleep(10)
    
    drone_manager.land()
    drone_manager.stop()

OSError: [WinError 10049] 요청한 주소는 해당 컨텍스트에서 유효하지 않습니다

## Drone의 상하좌우 이동

In [33]:
# 드론의 기본 움직임 거리 설정
DEFAULT_DISTANCE = 0.30

# 드론을 관리하기 위한 클래스 

class DroneManager(object):
    def __init__(self, host_ip = '192.168.10.2', host_port = 8889,
                drone_ip = '192.168.10.1', drone_port = 8889,
                is_imperial = False):
        self.host_ip = host_ip
        self.host_port = host_port
        self.drone_ip = drone_ip
        self.drone_port = drone_port
        self.drone_address = (drone_ip, drone_port)
        # 단위계를 영국단위계를 사용하는지 확인
        self.is_imperial = is_imperial 
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.socket.bind((self.host_ip, self.host_port))
        
        # 반응에 대한 default 값
        self.response = None
        # 어떤 이벤트로 멈추고 싶을때 사용할 이벤트 설정
        self.stop_event = threading.Event()
        # 스레드가 돌릴 함수 : receive_response이며 인자는 args로 tuple의 형태로 stopevent를 전달
        self._response_thread = threading.Thread(target = self.receive_response, args = (self.stop_event,))
        # 스레드 실행
        self._response_thread.start()
        
        # 어떤 기능을 수행하기 위해서는 command라는 명령이 먼저 Drone에 전달 되어야한다.
        self.socket.sendto(b'command', self.drone_address)
        self.socket.sendto(b'streamon',self.drone_address)
        
    # 스레딩 돌릴 함수( 해당 함수는 stop_event가 아닌 경우 계속해서 돌면서 정보를 받는다)
    def receive_response(self, stop_event):
        while not stop_event.is_set():
            try:
                self.response, ip = self.socket.recvfrom(3000)
                logger.info({'action' : 'receive_response' , 'response' : self.response})
            except socket.error as ex:
                logger.error({'action' : 'receive_response', 'ex' : ex})
                break
                
    # 클래스가 메모리에서 삭제될때 하는 메직메소드
    def __dell__(self):
        self.stop()
    
    # 클래스가 삭제된 경우 현재 열고 있던 소캣을 닫아준다.
    def stop(self):
        # 드론이 멈춘다면 해당 스레드 또한 종료시켜라
        self.stop_event.set()
        
        # receive_response에 while문이 실행 중이라면 종료 후 실행시키기 위한 구문
        retry = 0
        while self._response_thread.isAlive():
            time.sleep(0.3)
            if retry > 30:
                break
            # 너무 오랜시간을 기다리지 않기 위한 retry 값 수정
            retry += 1
        
        self.socket.close()
    
    # 이착륙과 같은 명령을 수향하기 위한 함수
    def send_command(self, command):
        # 로그 작성
        logger.info({'action' : 'send_command', 'command' : command}) 
        
        # 문자열로 명령어(command)가 들어오기 때문에 인코딩 후 통해 전달
        self.socket.sendto(command.encode('utf-8'),self.drone_address)
        
        retry = 0
        while self.response is None:
            time.sleep(0.3)
            if retry > 3:
                break
            retry += 1
        
        if self.response is None:
            response = None
        else:
            response = self.response.deconde('utf-8')
        self.response = None
        return response
        
    def takeoff(self):
        # takeoff의 수행여부 판단을 위해 return을 사용
        return self.send_command('takeoff')
        
    def land(self):
        # land의 수행여부 판단을 위해 return을 사용
        return self.send_command('land')
    
    # 거리와 원하는 방향을 입력받으면 해당 방향과 거리로 움직이기 위한 함수
    def move(self, direction, distance):
        distance = float(distance)
        if self.is_imperial:
            distance = int(round(distance * 30.48))
        else:
            distance = int(round(distance * 100))
        return self.send_command(f'{direction} {distance}')
    
    # 입력받은 명령이 up인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('up', distance)
    
    # 입력받은 명령이 down인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('down', distance)
    
    # 입력받은 명령이 left인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('left', distance)
    
    # 입력받은 명령이 right인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('right', distance)
    
    # 입력받은 명령이 forward인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('forward', distance)
    
    # 입력받은 명령이 back인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('back', distance)
    
    

In [34]:
# Python 코드가 처음에 불려왔을 때 실행
if __name__ == '__main__':
    drone_manager = DroneManager()
    drone_manager.takeoff()
    
    time.sleep(10)
    
    drone_manager.forward(0.30)
    time.sleep(5)
    drone_manager.right(0.30)
    time.sleep(5)
    drone_manager.back(0.30)
    time.sleep(5)
    drone_manager.left(0.30)
    time.sleep(5)
    drone_manager.up(0.30)
    time.sleep(5)
    drone_manager.down(0.30)
    time.sleep(5)

    drone_manager.land()
    drone_manager.stop()

OSError: [WinError 10049] 요청한 주소는 해당 컨텍스트에서 유효하지 않습니다

## 드론의 속도조절

In [35]:
 # 드론의 기본 움직임 거리 설정
DEFAULT_DISTANCE = 0.30

# 드론의 속도 조절을 위한 기본 설정
DEFAULT_SPEED = 10

# 드론을 관리하기 위한 클래스 

class DroneManager(object):
    def __init__(self, host_ip = '192.168.10.2', host_port = 8889,
                drone_ip = '192.168.10.1', drone_port = 8889,
                is_imperial = False, speed = DEFAULT_SPEED):
        self.host_ip = host_ip
        self.host_port = host_port
        self.drone_ip = drone_ip
        self.drone_port = drone_port
        self.drone_address = (drone_ip, drone_port)
        # 단위계를 영국단위계를 사용하는지 확인
        self.is_imperial = is_imperial 
        
        # 드론의 이동속도
        self.speed = speed
        
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.socket.bind((self.host_ip, self.host_port))
        
        # 반응에 대한 default 값
        self.response = None
        # 어떤 이벤트로 멈추고 싶을때 사용할 이벤트 설정
        self.stop_event = threading.Event()
        # 스레드가 돌릴 함수 : receive_response이며 인자는 args로 tuple의 형태로 stopevent를 전달
        self._response_thread = threading.Thread(target = self.receive_response, args = (self.stop_event,))
        # 스레드 실행
        self._response_thread.start()
        
        # 어떤 기능을 수행하기 위해서는 command라는 명령이 먼저 Drone에 전달 되어야한다.
        self.socket.sendto(b'command', self.drone_address)
        self.socket.sendto(b'streamon',self.drone_address)
        
        self.set_speed(self.speed)
        
    # 스레딩 돌릴 함수( 해당 함수는 stop_event가 아닌 경우 계속해서 돌면서 정보를 받는다)
    def receive_response(self, stop_event):
        while not stop_event.is_set():
            try:
                self.response, ip = self.socket.recvfrom(3000)
                logger.info({'action' : 'receive_response' , 'response' : self.response})
            except socket.error as ex:
                logger.error({'action' : 'receive_response', 'ex' : ex})
                break
                
    # 클래스가 메모리에서 삭제될때 하는 메직메소드
    def __dell__(self):
        self.stop()
    
    # 클래스가 삭제된 경우 현재 열고 있던 소캣을 닫아준다.
    def stop(self):
        # 드론이 멈춘다면 해당 스레드 또한 종료시켜라
        self.stop_event.set()
        
        # receive_response에 while문이 실행 중이라면 종료 후 실행시키기 위한 구문
        retry = 0
        while self._response_thread.isAlive():
            time.sleep(0.3)
            if retry > 30:
                break
            # 너무 오랜시간을 기다리지 않기 위한 retry 값 수정
            retry += 1
        
        self.socket.close()
    
    # 이착륙과 같은 명령을 수향하기 위한 함수
    def send_command(self, command):
        # 로그 작성
        logger.info({'action' : 'send_command', 'command' : command}) 
        
        # 문자열로 명령어(command)가 들어오기 때문에 인코딩 후 통해 전달
        self.socket.sendto(command.encode('utf-8'),self.drone_address)
        
        retry = 0
        while self.response is None:
            time.sleep(0.3)
            if retry > 3:
                break
            retry += 1
        
        if self.response is None:
            response = None
        else:
            response = self.response.deconde('utf-8')
        self.response = None
        return response
        
    def takeoff(self):
        # takeoff의 수행여부 판단을 위해 return을 사용
        return self.send_command('takeoff')
        
    def land(self):
        # land의 수행여부 판단을 위해 return을 사용
        return self.send_command('land')
    
    # 거리와 원하는 방향을 입력받으면 해당 방향과 거리로 움직이기 위한 함수
    def move(self, direction, distance):
        distance = float(distance)
        if self.is_imperial:
            distance = int(round(distance * 30.48))
        else:
            distance = int(round(distance * 100))
        return self.send_command(f'{direction} {distance}')
    
    # 입력받은 명령이 up인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('up', distance)
    
    # 입력받은 명령이 down인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('down', distance)
    
    # 입력받은 명령이 left인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('left', distance)
    
    # 입력받은 명령이 right인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('right', distance)
    
    # 입력받은 명령이 forward인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('forward', distance)
    
    # 입력받은 명령이 back인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('back', distance)
    
    # 드론의 움직이는 속도 조절
    def set_speed(self, speed):
        return self.send_command(f'speed {speed}')
    
    

In [36]:
# Python 코드가 처음에 불려왔을 때 실행
if __name__ == '__main__':
    drone_manager = DroneManager()
    drone_manager.takeoff()
    
    time.sleep(10)
    
    drone_manager.forward(0.30)
    time.sleep(5)
    drone_manager.right(0.30)
    time.sleep(5)
    drone_manager.back(0.30)
    time.sleep(5)
    drone_manager.left(0.30)
    time.sleep(5)
    
    drone_manager.set_speed(10)
    time.sleep(1)
    
    drone_manager.up(0.30)
    time.sleep(5)
    drone_manager.down(0.30)
    time.sleep(5)

    drone_manager.land()
    drone_manager.stop()

OSError: [WinError 10049] 요청한 주소는 해당 컨텍스트에서 유효하지 않습니다

## 드론 회전시키기

In [40]:
 # 드론의 기본 움직임 거리 설정
DEFAULT_DISTANCE = 0.30

# 드론의 속도 조절을 위한 기본 설정
DEFAULT_SPEED = 10

# 드론이 회전할때의 기본 각도 값 설정
DEFAULT_DEGREE = 10


# 드론을 관리하기 위한 클래스 

class DroneManager(object):
    def __init__(self, host_ip = '192.168.10.2', host_port = 8889,
                drone_ip = '192.168.10.1', drone_port = 8889,
                is_imperial = False, speed = DEFAULT_SPEED):
        self.host_ip = host_ip
        self.host_port = host_port
        self.drone_ip = drone_ip
        self.drone_port = drone_port
        self.drone_address = (drone_ip, drone_port)
        # 단위계를 영국단위계를 사용하는지 확인
        self.is_imperial = is_imperial 
        
        # 드론의 이동속도
        self.speed = speed
        
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.socket.bind((self.host_ip, self.host_port))
        
        # 반응에 대한 default 값
        self.response = None
        # 어떤 이벤트로 멈추고 싶을때 사용할 이벤트 설정
        self.stop_event = threading.Event()
        # 스레드가 돌릴 함수 : receive_response이며 인자는 args로 tuple의 형태로 stopevent를 전달
        self._response_thread = threading.Thread(target = self.receive_response, args = (self.stop_event,))
        # 스레드 실행
        self._response_thread.start()
        
        # 어떤 기능을 수행하기 위해서는 command라는 명령이 먼저 Drone에 전달 되어야한다.
        self.socket.sendto(b'command', self.drone_address)
        self.socket.sendto(b'streamon',self.drone_address)
        
        self.set_speed(self.speed)
        
    # 스레딩 돌릴 함수( 해당 함수는 stop_event가 아닌 경우 계속해서 돌면서 정보를 받는다)
    def receive_response(self, stop_event):
        while not stop_event.is_set():
            try:
                self.response, ip = self.socket.recvfrom(3000)
                logger.info({'action' : 'receive_response' , 'response' : self.response})
            except socket.error as ex:
                logger.error({'action' : 'receive_response', 'ex' : ex})
                break
                
    # 클래스가 메모리에서 삭제될때 하는 메직메소드
    def __dell__(self):
        self.stop()
    
    # 클래스가 삭제된 경우 현재 열고 있던 소캣을 닫아준다.
    def stop(self):
        # 드론이 멈춘다면 해당 스레드 또한 종료시켜라
        self.stop_event.set()
        
        # receive_response에 while문이 실행 중이라면 종료 후 실행시키기 위한 구문
        retry = 0
        while self._response_thread.isAlive():
            time.sleep(0.3)
            if retry > 30:
                break
            # 너무 오랜시간을 기다리지 않기 위한 retry 값 수정
            retry += 1
        
        self.socket.close()
    
    # 이착륙과 같은 명령을 수향하기 위한 함수
    def send_command(self, command):
        # 로그 작성
        logger.info({'action' : 'send_command', 'command' : command}) 
        
        # 문자열로 명령어(command)가 들어오기 때문에 인코딩 후 통해 전달
        self.socket.sendto(command.encode('utf-8'),self.drone_address)
        
        retry = 0
        while self.response is None:
            time.sleep(0.3)
            if retry > 3:
                break
            retry += 1
        
        if self.response is None:
            response = None
        else:
            response = self.response.deconde('utf-8')
        self.response = None
        return response
        
    def takeoff(self):
        # takeoff의 수행여부 판단을 위해 return을 사용
        return self.send_command('takeoff')
        
    def land(self):
        # land의 수행여부 판단을 위해 return을 사용
        return self.send_command('land')
    
    # 거리와 원하는 방향을 입력받으면 해당 방향과 거리로 움직이기 위한 함수
    def move(self, direction, distance):
        distance = float(distance)
        if self.is_imperial:
            distance = int(round(distance * 30.48))
        else:
            distance = int(round(distance * 100))
        return self.send_command(f'{direction} {distance}')
    
    # 입력받은 명령이 up인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('up', distance)
    
    # 입력받은 명령이 down인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('down', distance)
    
    # 입력받은 명령이 left인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('left', distance)
    
    # 입력받은 명령이 right인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('right', distance)
    
    # 입력받은 명령이 forward인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('forward', distance)
    
    # 입력받은 명령이 back인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('back', distance)
    
    # 드론의 움직이는 속도 조절
    def set_speed(self, speed):
        return self.send_command(f'speed {speed}')
    
    # 드론이 바라보는 방향을 시계방향으로 회전
    def clockwise(self, degree = DEFAULT_DEGREE):
        return self.send_command(f'cw {degree}')
    
    # 드론이 바라보는 방향을 반시계방향으로 회전
    def counter_clockwise(self, degree = DEFAULT_DEGREE):
        return self.send_command(f'ccw {degree}')

In [41]:
# Python 코드가 처음에 불려왔을 때 실행
if __name__ == '__main__':
    drone_manager = DroneManager()
    drone_manager.takeoff()
    
    time.sleep(10)
    
    drone_manager.clockwise(90)
    time.sleep(5)
    drone_manager.counter_clockwise(90)
    time.sleep(5)
    
    drone_manager.forward(0.30)
    time.sleep(5)
    drone_manager.right(0.30)
    time.sleep(5)
    drone_manager.back(0.30)
    time.sleep(5)
    drone_manager.left(0.30)
    time.sleep(5)
    
    drone_manager.set_speed(10)
    time.sleep(1)
    
    drone_manager.up(0.30)
    time.sleep(5)
    drone_manager.down(0.30)
    time.sleep(5)

    drone_manager.land()
    drone_manager.stop()

OSError: [WinError 10049] 요청한 주소는 해당 컨텍스트에서 유효하지 않습니다

## 드론 플립

In [42]:
 # 드론의 기본 움직임 거리 설정
DEFAULT_DISTANCE = 0.30

# 드론의 속도 조절을 위한 기본 설정
DEFAULT_SPEED = 10

# 드론이 회전할때의 기본 각도 값 설정
DEFAULT_DEGREE = 10


# 드론을 관리하기 위한 클래스 

class DroneManager(object):
    def __init__(self, host_ip = '192.168.10.2', host_port = 8889,
                drone_ip = '192.168.10.1', drone_port = 8889,
                is_imperial = False, speed = DEFAULT_SPEED):
        self.host_ip = host_ip
        self.host_port = host_port
        self.drone_ip = drone_ip
        self.drone_port = drone_port
        self.drone_address = (drone_ip, drone_port)
        # 단위계를 영국단위계를 사용하는지 확인
        self.is_imperial = is_imperial 
        
        # 드론의 이동속도
        self.speed = speed
        
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.socket.bind((self.host_ip, self.host_port))
        
        # 반응에 대한 default 값
        self.response = None
        # 어떤 이벤트로 멈추고 싶을때 사용할 이벤트 설정
        self.stop_event = threading.Event()
        # 스레드가 돌릴 함수 : receive_response이며 인자는 args로 tuple의 형태로 stopevent를 전달
        self._response_thread = threading.Thread(target = self.receive_response, args = (self.stop_event,))
        # 스레드 실행
        self._response_thread.start()
        
        # 어떤 기능을 수행하기 위해서는 command라는 명령이 먼저 Drone에 전달 되어야한다.
        self.socket.sendto(b'command', self.drone_address)
        self.socket.sendto(b'streamon',self.drone_address)
        
        self.set_speed(self.speed)
        
    # 스레딩 돌릴 함수( 해당 함수는 stop_event가 아닌 경우 계속해서 돌면서 정보를 받는다)
    def receive_response(self, stop_event):
        while not stop_event.is_set():
            try:
                self.response, ip = self.socket.recvfrom(3000)
                logger.info({'action' : 'receive_response' , 'response' : self.response})
            except socket.error as ex:
                logger.error({'action' : 'receive_response', 'ex' : ex})
                break
                
    # 클래스가 메모리에서 삭제될때 하는 메직메소드
    def __dell__(self):
        self.stop()
    
    # 클래스가 삭제된 경우 현재 열고 있던 소캣을 닫아준다.
    def stop(self):
        # 드론이 멈춘다면 해당 스레드 또한 종료시켜라
        self.stop_event.set()
        
        # receive_response에 while문이 실행 중이라면 종료 후 실행시키기 위한 구문
        retry = 0
        while self._response_thread.isAlive():
            time.sleep(0.3)
            if retry > 30:
                break
            # 너무 오랜시간을 기다리지 않기 위한 retry 값 수정
            retry += 1
        
        self.socket.close()
    
    # 이착륙과 같은 명령을 수향하기 위한 함수
    def send_command(self, command):
        # 로그 작성
        logger.info({'action' : 'send_command', 'command' : command}) 
        
        # 문자열로 명령어(command)가 들어오기 때문에 인코딩 후 통해 전달
        self.socket.sendto(command.encode('utf-8'),self.drone_address)
        
        retry = 0
        while self.response is None:
            time.sleep(0.3)
            if retry > 3:
                break
            retry += 1
        
        if self.response is None:
            response = None
        else:
            response = self.response.deconde('utf-8')
        self.response = None
        return response
        
    def takeoff(self):
        # takeoff의 수행여부 판단을 위해 return을 사용
        return self.send_command('takeoff')
        
    def land(self):
        # land의 수행여부 판단을 위해 return을 사용
        return self.send_command('land')
    
    # 거리와 원하는 방향을 입력받으면 해당 방향과 거리로 움직이기 위한 함수
    def move(self, direction, distance):
        distance = float(distance)
        if self.is_imperial:
            distance = int(round(distance * 30.48))
        else:
            distance = int(round(distance * 100))
        return self.send_command(f'{direction} {distance}')
    
    # 입력받은 명령이 up인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('up', distance)
    
    # 입력받은 명령이 down인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('down', distance)
    
    # 입력받은 명령이 left인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('left', distance)
    
    # 입력받은 명령이 right인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('right', distance)
    
    # 입력받은 명령이 forward인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('forward', distance)
    
    # 입력받은 명령이 back인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('back', distance)
    
    # 드론의 움직이는 속도 조절
    def set_speed(self, speed):
        return self.send_command(f'speed {speed}')
    
    # 드론이 바라보는 방향을 시계방향으로 회전
    def clockwise(self, degree = DEFAULT_DEGREE):
        return self.send_command(f'cw {degree}')
    
    # 드론이 바라보는 방향을 반시계방향으로 회전
    def counter_clockwise(self, degree = DEFAULT_DEGREE):
        return self.send_command(f'ccw {degree}')
    
    # 드론을 원하는 방향으로 flip 하기 위한 메소드들
    def flip_forward(self):
        return self.send_command(f'flip f')
    
    def flip_back(self):
        return self.send_command(f'flip b')
    
    def flip_right(self):
        return self.send_command(f'flip r')
    
    def flip_left(self):
        return self.send_command(f'flip l')
    

In [43]:
# Python 코드가 처음에 불려왔을 때 실행
if __name__ == '__main__':
    drone_manager = DroneManager()
    drone_manager.takeoff()
    
    time.sleep(10)
    
    drone_manager.clockwise(90)
    time.sleep(5)
    drone_manager.counter_clockwise(90)
    time.sleep(5)
    
    drone_manager.forward(0.30)
    time.sleep(5)
    drone_manager.right(0.30)
    time.sleep(5)
    drone_manager.back(0.30)
    time.sleep(5)
    drone_manager.left(0.30)
    time.sleep(5)
    
    drone_manager.set_speed(10)
    time.sleep(1)
    
    drone_manager.up(0.30)
    time.sleep(5)
    drone_manager.down(0.30)
    time.sleep(5)
    
    drone_manager.flip_forward()
    time.sleep(5)
    drone_manager.flip_back()
    time.sleep(5)
    drone_manager.flip_right()
    time.sleep(5)
    drone_manager.flip_left()
    time.sleep(5)
    
    drone_manager.land()
    drone_manager.stop()

OSError: [WinError 10049] 요청한 주소는 해당 컨텍스트에서 유효하지 않습니다

## 드론 순찰

In [45]:
import contextlib

import logging
import socket
import sys
import time

# 언제 드론에서 컴퓨터로 정보가 전달될지 모르기 때문에 스레딩 사용
import threading

In [ ]:
 # 드론의 기본 움직임 거리 설정
DEFAULT_DISTANCE = 0.30

# 드론의 속도 조절을 위한 기본 설정
DEFAULT_SPEED = 10

# 드론이 회전할때의 기본 각도 값 설정
DEFAULT_DEGREE = 10


# 드론을 관리하기 위한 클래스 

class DroneManager(object):
    def __init__(self, host_ip = '192.168.10.2', host_port = 8889,
                drone_ip = '192.168.10.1', drone_port = 8889,
                is_imperial = False, speed = DEFAULT_SPEED):
        self.host_ip = host_ip
        self.host_port = host_port
        self.drone_ip = drone_ip
        self.drone_port = drone_port
        self.drone_address = (drone_ip, drone_port)
        # 단위계를 영국단위계를 사용하는지 확인
        self.is_imperial = is_imperial 
        
        # 드론의 이동속도
        self.speed = speed
        
        self.socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        self.socket.bind((self.host_ip, self.host_port))
        
        # 반응에 대한 default 값
        self.response = None
        # 어떤 이벤트로 멈추고 싶을때 사용할 이벤트 설정
        self.stop_event = threading.Event()
        # 스레드가 돌릴 함수 : receive_response이며 인자는 args로 tuple의 형태로 stopevent를 전달
        self._response_thread = threading.Thread(target = self.receive_response, args = (self.stop_event,))
        # 스레드 실행
        self._response_thread.start()
        
        # 순찰 기능을 수행중인지 확인하는 변수
        self.patrol_event = None
        self.is_patrol = False
        
        # 순찰기능을 수행하는 스레드 1개를 세마포어를 이용하여 제어
        self._patrol_semaphore = threading.Semaphore(1)
        self._thread_patrol = None
        
        
        
        # 어떤 기능을 수행하기 위해서는 command라는 명령이 먼저 Drone에 전달 되어야한다.
        self.socket.sendto(b'command', self.drone_address)
        self.socket.sendto(b'streamon',self.drone_address)
        
        self.set_speed(self.speed)
        
    # 스레딩 돌릴 함수( 해당 함수는 stop_event가 아닌 경우 계속해서 돌면서 정보를 받는다)
    def receive_response(self, stop_event):
        while not stop_event.is_set():
            try:
                self.response, ip = self.socket.recvfrom(3000)
                logger.info({'action' : 'receive_response' , 'response' : self.response})
            except socket.error as ex:
                logger.error({'action' : 'receive_response', 'ex' : ex})
                break
                
    # 클래스가 메모리에서 삭제될때 하는 메직메소드
    def __dell__(self):
        self.stop()
    
    # 클래스가 삭제된 경우 현재 열고 있던 소캣을 닫아준다.
    def stop(self):
        # 드론이 멈춘다면 해당 스레드 또한 종료시켜라
        self.stop_event.set()
        
        # receive_response에 while문이 실행 중이라면 종료 후 실행시키기 위한 구문
        retry = 0
        while self._response_thread.isAlive():
            time.sleep(0.3)
            if retry > 30:
                break
            # 너무 오랜시간을 기다리지 않기 위한 retry 값 수정
            retry += 1
        
        self.socket.close()
    
    # 이착륙과 같은 명령을 수향하기 위한 함수
    def send_command(self, command):
        # 로그 작성
        logger.info({'action' : 'send_command', 'command' : command}) 
        
        # 문자열로 명령어(command)가 들어오기 때문에 인코딩 후 통해 전달
        self.socket.sendto(command.encode('utf-8'),self.drone_address)
        
        retry = 0
        while self.response is None:
            time.sleep(0.3)
            if retry > 3:
                break
            retry += 1
        
        if self.response is None:
            response = None
        else:
            response = self.response.deconde('utf-8')
        self.response = None
        return response
        
    def takeoff(self):
        # takeoff의 수행여부 판단을 위해 return을 사용
        return self.send_command('takeoff')
        
    def land(self):
        # land의 수행여부 판단을 위해 return을 사용
        return self.send_command('land')
    
    # 거리와 원하는 방향을 입력받으면 해당 방향과 거리로 움직이기 위한 함수
    def move(self, direction, distance):
        distance = float(distance)
        if self.is_imperial:
            distance = int(round(distance * 30.48))
        else:
            distance = int(round(distance * 100))
        return self.send_command(f'{direction} {distance}')
    
    # 입력받은 명령이 up인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('up', distance)
    
    # 입력받은 명령이 down인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('down', distance)
    
    # 입력받은 명령이 left인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('left', distance)
    
    # 입력받은 명령이 right인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('right', distance)
    
    # 입력받은 명령이 forward인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('forward', distance)
    
    # 입력받은 명령이 back인 경우 
    def up(self, distance = DEFAULT_DISTANCE):
        return self.move('back', distance)
    
    # 드론의 움직이는 속도 조절
    def set_speed(self, speed):
        return self.send_command(f'speed {speed}')
    
    # 드론이 바라보는 방향을 시계방향으로 회전
    def clockwise(self, degree = DEFAULT_DEGREE):
        return self.send_command(f'cw {degree}')
    
    # 드론이 바라보는 방향을 반시계방향으로 회전
    def counter_clockwise(self, degree = DEFAULT_DEGREE):
        return self.send_command(f'ccw {degree}')
    
    # 드론을 원하는 방향으로 flip 하기 위한 메소드들
    def flip_forward(self):
        return self.send_command(f'flip f')
    
    def flip_back(self):
        return self.send_command(f'flip b')
    
    def flip_right(self):
        return self.send_command(f'flip r')
    
    def flip_left(self):
        return self.send_command(f'flip l')
    
    # 순찰기능을 수행하는 스레드를 실행시키는 메소드
    def patrol(self):
        if not self.is_patrol:
            self.patrol_event = threading.Event()
            self._thread_patrol = threading.Thread(target = self._patrol,
                                                  args = (self._patrol_semaphore, self.patrol_event,))
            self._thread_patrol.start()
            self.is_patrol = True
            
    #  patrol에서 실행시키기 위해 필요한 메소드
    def _patrol(self, semaphore, stop_event):
        is_acquire = semaphore.acquire(blocking = False)
        if is_acquire:
            logger.info({'action' : '_patrol', 'status' : 'acquire'})
            with contextlib.ExitStack() as stack:
                stack.callback(semaphore.release)
                status = 0
                while not stop_event.is_set():
                    if status == 1:
                        self.up()
                    if status == 2:
                        self.clockwise(2)
                    if status == 3:
                        self.down()
                    if status == 4:
                        status = 0
                    time.sleep(5)
        else:
            logger.warning({'action' : '_patrol', 'status' : 'not_acquire'})
            
    # 순찰을 멈추는 기능
    def stop_patrol(self):
        if self.is_patrol:
            self.patrol_event.set()
            retry = 0
            while self._thread_patrol.isAlive():
                time.sleep(0.3)
                if retry > 300:
                    break
                retry += 1
            self.is_patrol = False
            
    

In [49]:
if __name__ == '__main__':
    drone_manager = DroneManager()
    
    drone_manager.takeoff()
    time.sleep(10)
    drone_manager.patrol()
    time.sleep(45)
    drone_manager.stop_patrol()
    time.sleep(5)
    drone_manager.land()
    drone_manager.stop()

OSError: [WinError 10049] 요청한 주소는 해당 컨텍스트에서 유효하지 않습니다